In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy.spatial
import pandas as pd
import sklearn.decomposition
import matplotlib.pyplot as plt
# import keras
from sklearn import preprocessing,linear_model
from sklearn.metrics import pairwise_distances,mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from utils.readProfiles import *
from utils.pred_models import *
# from yellowbrick.model_selection import FeatureImportances

sns.set_style("whitegrid")
# from utils import networksEvol, tsne, readProfiles
# import umap

##### create single blue colormap to change defualt colors in yellowbrick
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
blue_cmap = cm.get_cmap('Blues', 200)
single_blue_cmap=ListedColormap(blue_cmap(np.linspace(0.7, 0.9, 20)))

### In this notebook you can find your answer to these questions:


1. What CP features are related to your Gene under study?
2. What are the Genes invloved in predicting a CP feature?


* Due to quality of overall predictions as reported in the paper, we use:
   - **LINCS** dataset for chemical perturbation reference 
   - **LUAD** dataset for genetic perturbation reference 


## User input parameters

In [2]:
# In which perturbation context you want to explore your features
# "genetic" or "chemical"
which_dataset_to_use="genetic"
profileType = "normalized_variable_selected"

cp_features_under_test=['Cells_Texture_InfoMeas1_RNA_3_0']
# Cells_Texture_InfoMeas1_RNA_3_0,Cytoplasm_AreaShape_Zernike_9_3
ge_features_under_test=['CISD1']

#### Paths

In [3]:
# procProf_dir='/home/ubuntu/datasetsbucket/Rosetta-GE-CP/'
# procProf_dir='/home/ubuntu/bucket/projects/2018_04_20_Rosetta/workspace/'
procProf_dir = "/Users/spasovs/Documents/cpg0003-rosetta/broad/workspace"
results_dir='./results/'


## What are the CP features invloved in predicting a specific gene?

In [4]:
if which_dataset_to_use=="genetic":
    dataset='LUAD';
elif which_dataset_to_use=="chemical":
    dataset='LINCS';    


filter_perts='highRepOverlap'
# filter_perts=''
repCorrFilePath='./results/RepCor/RepCorrDF.xlsx'

filter_repCorr_params=[filter_perts,repCorrFilePath]    
    
################################################
pertColName='PERT'

mergProf_treatLevel,cp_features,l1k_features = \
read_paired_treatment_level_profiles(procProf_dir,dataset,profileType,filter_repCorr_params,1)


l1k=mergProf_treatLevel[[pertColName]+l1k_features]
cp=mergProf_treatLevel[[pertColName]+cp_features]


if dataset=='LINCS':     
    cp.loc[:,'Compounds']=cp.loc[:,'PERT'].str[0:13]
    l1k.loc[:,'Compounds']=l1k.loc[:,'PERT'].str[0:13]
else:
    cp.loc[:,'Compounds']=cp.loc[:,'PERT'].values
    l1k.loc[:,'Compounds']=l1k.loc[:,'PERT'].values


scaler_ge = preprocessing.StandardScaler()
scaler_cp = preprocessing.StandardScaler()
l1k_scaled=l1k.copy()
l1k_scaled[l1k_features] = scaler_ge.fit_transform(l1k[l1k_features].values)
cp_scaled=cp.copy()
cp_scaled[cp_features] = scaler_cp.fit_transform(cp[cp_features].values.astype('float64'))


if 1:
    cp=cp_scaled.copy()
    l1k=l1k_scaled.copy()

##############################

l1k,l1k_features_gn=rename_affyprobe_to_genename(l1k,l1k_features,'./idmap.xlsx')

# for g in ge_features_under_test:
#     viz = FeatureImportances(linear_model.Lasso(),relative=True, topn=20)
#     viz = FeatureImportances(linear_model.Lasso(alpha=0.001), labels=l1k_features_gn, relative=False,topn=20)
#     print(g)
#     model = linear_model.LinearRegression()
#     model = linear_model.Lasso(alpha=0.02)
#     model.fit(cp[cp_features],l1k[g])
# #     model = RandomForestClassifier(n_estimators=10),colormap=single_blue_cmap
#     viz = FeatureImportances(model,topn=20,colormap='coolwarm')    
#     viz = FeatureImportances(model,topn=20,colormap=single_blue_cmap)    
    
#     viz.fit(cp[cp_features],l1k[g])
#     viz.show()


/Users/spasovs/PycharmProjects/2022_Haghighi_NatureMethods/utils/readProfiles.py:54: DtypeWarning: Columns (1023,1028,1032) have mixed types. Specify dtype option on import or set low_memory=False.
  l1k_data_repLevel = pd.read_csv(dataDir + "/L1000/replicate_level_l1k.csv.gz")


LUAD: Replicate Level Shapes (nSamples x nFeatures): cp:  6144 , 291 ,  l1k:  4232 , 978
l1k n of rep:  8.0
cp n of rep:  8.0
CP: from  593  to  364
l1k: from  529  to  275
CP and l1k high rep overlap:  197
Treatment Level Shapes (nSamples x nFeatures+metadata): (198, 292) (198, 979) Merged Profiles Shape: (198, 1270)


/var/folders/04/zvv56jd16clbkldmdbbxgtxh0lzgcm/T/ipykernel_3558/1711028465.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cp.loc[:,'Compounds']=cp.loc[:,'PERT'].values
/var/folders/04/zvv56jd16clbkldmdbbxgtxh0lzgcm/T/ipykernel_3558/1711028465.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l1k.loc[:,'Compounds']=l1k.loc[:,'PERT'].values


In [21]:
l1k

,PERT,AARS1,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,...,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10,Compounds
0,ABCB9_WT.c,1.257226,1.010581,-0.072920,1.063642,0.531769,0.218904,-0.480550,-0.786553,-1.435156,...,-0.139313,0.496745,1.492526,1.581734,0.258691,1.101618,-0.402966,-0.266990,-1.482962,ABCB9_WT.c
1,ABCB9_WT.o,1.133482,0.874074,-0.198010,3.045103,-2.553283,-0.862889,0.500027,0.336615,-1.592294,...,-1.317525,0.765131,0.553405,2.283294,2.267496,1.177028,-0.043846,0.860563,0.038179,ABCB9_WT.o
2,ABCB9_p.R281L,0.654616,0.769779,-1.054488,-0.553884,1.019900,-0.648544,-0.363894,-0.837984,0.339467,...,0.679489,-0.903800,-0.741984,0.157314,-0.774862,-0.054116,-0.069247,-0.340132,-0.217657,ABCB9_p.R281L
3,ABCB9_p.V140M,1.353112,0.439655,0.056872,0.603507,1.487743,-0.485399,0.113293,0.523607,0.299900,...,0.079894,0.358964,-0.334741,0.069632,-0.520208,0.088992,-0.283913,-0.891144,-0.183542,ABCB9_p.V140M
4,ACAA1_WT.o,0.006640,-0.272687,-1.054505,-0.885362,-1.569519,-0.168795,-0.037172,0.923348,16.168128,...,0.898372,0.510917,-0.053053,-0.353098,0.212389,0.102215,-0.314048,-1.074330,-0.569681,ACAA1_WT.o
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,ZNF597_p.F16L,-1.024328,-0.660076,-0.133872,-0.614384,-1.104568,1.135229,-0.883238,2.311535,0.696120,...,1.124275,0.790209,0.111522,-0.478692,0.475747,-1.361167,-1.420489,-0.859104,0.093447,ZNF597_p.F16L
522,ZNF597_p.L283V,-1.509601,-0.379900,-0.522433,-0.665734,-0.903720,2.031702,-0.736583,0.941593,0.943498,...,2.389745,0.185403,-0.980281,-1.995654,-0.143534,-1.319740,-0.918101,-1.539747,-0.642420,ZNF597_p.L283V
523,ZNF597_p.R247T,2.061479,0.325048,0.267462,1.953930,0.538648,-0.601433,0.400943,0.311227,-0.357839,...,-0.850908,0.688712,0.621986,1.556714,0.395243,0.984547,-0.506229,1.280846,-0.334325,ZNF597_p.R247T
524,ZNF597_p.V21L,-1.886986,-0.468054,-1.017953,-0.640910,-0.392879,1.694995,-0.557273,0.719272,0.696452,...,0.323209,-2.451159,-0.309703,-2.001804,0.057528,-0.736288,4.973821,-0.322936,0.071291,ZNF597_p.V21L


In [5]:
cols = set(l1k.columns)
inds = set(l1k['PERT'].str.split('_').str[0].values)

In [7]:
len(inds)

70

## What are the Genes invloved in predicting a CP feature?

In [3]:
if which_dataset_to_use=="genetic":
    dataset='LUAD';
elif which_dataset_to_use=="chemical":
    dataset='LINCS';    

filter_perts='highRepOverlap'
repCorrFilePath='./results/RepCor/RepCorrDF.xlsx'

filter_repCorr_params=[filter_perts,repCorrFilePath]        

################################################
pertColName='PERT'

mergedProfiles_treatLevel,cp_features,l1k_features = \
read_paired_treatment_level_profiles(procProf_dir,dataset,profileType,filter_repCorr_params,1)


l1k=mergedProfiles_treatLevel[[pertColName]+l1k_features]
cp=mergedProfiles_treatLevel[[pertColName]+cp_features]


if dataset=='LINCS':     
    cp.loc[:,'Compounds']=cp.loc[:,'PERT'].str[0:13]
    l1k.loc[:,'Compounds']=l1k.loc[:,'PERT'].str[0:13]
else:
    cp.loc[:,'Compounds']=cp.loc[:,'PERT'].values
    l1k.loc[:,'Compounds']=l1k.loc[:,'PERT'].values


scaler_ge = preprocessing.StandardScaler()
scaler_cp = preprocessing.StandardScaler()
l1k_scaled=l1k.copy()
l1k_scaled[l1k_features] = scaler_ge.fit_transform(l1k[l1k_features].values)
cp_scaled=cp.copy()
cp_scaled[cp_features] = scaler_cp.fit_transform(cp[cp_features].values.astype('float64'))

cp=cp_scaled.copy()
l1k=l1k_scaled.copy()

##############################
# meta=pd.read_csv("/home/ubuntu/bucket/projects/2018_04_20_Rosetta/workspace/metadata/affy_probe_gene_mapping.txt",delimiter="\t",header=None, names=["probe_id", "gene"])
# meta_gene_probID=meta.set_index('probe_id')
# d = dict(zip(meta_gene_probID.index, meta_gene_probID['gene']))
# l1k_features_gn=[d[l] for l in l1k_features]
# l1k = l1k.rename(columns=d)    

l1k,l1k_features_gn=rename_affyprobe_to_genename(l1k,l1k_features,'./idmap.xlsx')

    

# for c in cp_features_under_test:
#     print(c)
#     viz = FeatureImportances(linear_model.Lasso(),relative=True, topn=20)
#     viz = FeatureImportances(linear_model.Lasso(alpha=0.001), labels=l1k_features_gn, relative=False,topn=20)
    
#     model = linear_model.LinearRegression()
#     model.fit(l1k[l1k_features_gn],cp[c])
# #     model = RandomForestClassifier(n_estimators=10)

#     viz = FeatureImportances(model,topn=20,colormap=single_blue_cmap,size=(260, 360),\
#                              title="Top genes involved in\n prediction of feature\n'"+c+"'")    
#     viz.fit(l1k[l1k_features_gn],cp[c]);
#     viz.show()
#     viz.show(outpath='fig3c.pdf')

NameError: name 'which_dataset_to_use' is not defined

In [2]:
cols = set(l1k.columns)
inds = set(l1k['PERT'].str.split('_').str[0].values)

NameError: name 'l1k' is not defined

In [48]:
len(inds)

70

In [41]:
l1k['PERT']

0       BRD-A03506276_0.04
1       BRD-A03506276_0.12
2       BRD-A03506276_0.37
3       BRD-A03506276_1.11
4       BRD-A03506276_3.33
               ...        
1136    BRD-K99792991_1.11
1137    BRD-K99792991_3.33
1138    BRD-K99964838_3.33
1139    BRD-M80207679_3.33
1140                negcon
Name: PERT, Length: 1141, dtype: object

In [ ]:
from utils.saveAsNewSheetToExistingFile import saveAsNewSheetToExistingFile
source_data_add='./Figs_Source_Data.xlsx'
source_data_fig3c=pd.DataFrame(np.stack([viz.features_,viz.feature_importances_],axis=1),\
                            columns=['Genes','RCM']).sort_values(by='RCM',ascending=False).reset_index(drop=True)
if 1:
    viz.show(outpath='fig3c.pdf')
    saveAsNewSheetToExistingFile(source_data_add,source_data_fig3c,'fig3c')

In [1]:
# fig, ax = plt.subplots(figsize=(3,5))
# ax.set_title('fdg')
# # viz = FeatureImportances(model,topn=20,ax=ax)
# # viz.show()
# model = linear_model.LinearRegression()
# model.fit(l1k[l1k_features_gn],cp[c])
# # #     model = RandomForestClassifier(n_estimators=10)
# viz = FeatureImportances(model,topn=20,ax=ax)    

# viz.fit(l1k[l1k_features_gn],cp[c])
# viz.show()

In [2]:
# pwd

In [102]:
# %matplotlib inline
# viz = FeatureImportances(model,topn=20,stack=False,colormap=single_blue_cmap,size=(220, 350))    
# viz.ax.set_xlim([-100,100])
# viz.fit(l1k[l1k_features_gn],cp[c])
# viz.show()